In [1]:
import time
import warnings

import numpy as np
from numpy.linalg import norm

from pyROMs.pod import POD

from utils import *

warnings.filterwarnings('ignore')

Define Routines

In [5]:
def interpolant_study(dataset, params,
                      test_size, seed, interior_only):
    """Perform a mode truncation study."""
    vars = params["vars"]
    tau = params["tau"]
    interp = params["interp"]
    epsilon = params["epsilon"]

    # Setup storage
    out = {'interpolant': [],
           'mean': [], 'max': [], 'min': []}

    # Setup POD model
    pod = POD(svd_rank=0,
              method=interp,
              epsilon=epsilon)

    # Setup training/validation sets
    splits = dataset.train_test_split(
        variables=vars, test_size=test_size,
        seed=seed, interior_only=interior_only)
    X_train, X_test, Y_train, Y_test = splits

    # Define interpolants
    interpolants = [
        'rbf_linear', 'rbf_thin_plate_spline',
        'rbf_cubic', 'rbf_quintic', 'rbf_gaussian',
        'linear', 'nearest'
    ]

    # Run study
    print(f"Starting the interpolation study...")
    for interpolant in interpolants:

        # Construct POD model, predict test data
        pod.fit(X_train, Y_train, method=interpolant)
        X_pod = pod.predict(Y_test)

        # Format datasets
        X_pod = dataset.unstack_simulation_vector(X_pod)
        X_test = dataset.unstack_simulation_vector(X_test)

        # Compute simulation errors
        errors = np.zeros(len(X_test))
        for i in range(len(X_test)):
            errors[i] = norm(X_test[i]-X_pod[i])/norm(X_test[i])

        # Store results
        out['interpolant'].append(interpolant)
        out['mean'].append(np.mean(errors))
        out['max'].append(np.max(errors))
        out['min'].append(np.min(errors))
    print(f"Number of Snapshots:\t{pod.n_snapshots}")
    print(f"Number of Validations:\t{len(X_test)}")
    print(f"Number of POD Modes:\t{pod.n_modes}")
    return out


def print_table(results):
    """Print a LaTeX table with the results."""
    msg = "\\begin{tabular}{|c|c|c|c|}" \
      "\n\t\hline" \
      "\n\t\\textbf{Interpolation Method} & \\textbf{Mean Error} & " \
      "\\textbf{Max Error} & \\textbf{Min Error} \\\\ \hline"
    for i, interpolant in enumerate(results['interpolant']):
        interpolant = interpolant.split('_')
        if interpolant[0] == 'rbf':
            interpolant[0] = interpolant[0].upper()
            for w in range(len(interpolant[1:])):
                interpolant[w+1] = interpolant[w+1].capitalize()
            interpolant = " ".join(interpolant[1:]) + " RBF"
            if "Gaussian" in interpolant:
                epsilon = get_params(problem_name)["epsilon"]
                interpolant += f", $\\epsilon$ = {epsilon:.2e}"
        else:
            interpolant = interpolant[0].capitalize()
            if interpolant == "Nearest":
                interpolant += " Neighbor"
            elif interpolant == "Cubic":
                interpolant += " Spline"

        msg += f"\n\t\hline {interpolant} & " \
               f"{results['mean'][i]:.3e} & " \
               f"{results['max'][i]:.3e} & " \
               f"{results['min'][i]:.3e} \\\\"
    msg += "\n\t\hline\n\\end{tabular}"
    print()
    print(msg)

Parse the Data

In [3]:
problem_name = input("What problem? ")

print("Loading and formating the data...")
t_start = time.time()
dataset = get_data(problem_name)
t_end = time.time()
print(f"Loading the data took {t_end - t_start:3f} s")

Loading and formating the data...
Loading the data took 12.885374 s


Perform the Interpolation Study

In [9]:
params = get_params(problem_name)

test_size = 0.8
interior_only = True
seed = 12

results = interpolant_study(
    dataset, params, test_size, seed, interior_only)
print_table(results)



Starting the interpolation study...
Number of Snapshots:	57
Number of Validations:	7
Number of POD Modes:	27

\begin{tabular}{|c|c|c|c|}
	\hline
	\textbf{Interpolation Method} & \textbf{Mean Error} & \textbf{Max Error} & \textbf{Min Error} \\ \hline
	\hline Linear RBF & 1.990e-02 & 3.059e-02 & 8.016e-03 \\
	\hline Thin Plate Spline RBF & 6.799e-03 & 8.917e-03 & 4.455e-03 \\
	\hline Cubic RBF & 1.248e-02 & 1.816e-02 & 8.392e-03 \\
	\hline Quintic RBF & 1.274e-02 & 1.925e-02 & 3.534e-03 \\
	\hline Gaussian RBF, $\epsilon$ = 2.00e+01 & 6.224e-04 & 9.395e-04 & 2.524e-04 \\
	\hline Linear & 1.942e-02 & 4.451e-02 & 6.596e-03 \\
	\hline Nearest Neighbor & 6.563e-02 & 1.536e-01 & 2.955e-02 \\
	\hline
\end{tabular}
